In [ ]:
from diffusers import StableDiffusionPipeline, DiffusionPipeline, DDPMScheduler, UNet2DModel
import torch
from torch.utils.mobile_optimizer import optimize_for_mobile

In [ ]:
path = "stablediffusionapi/anything-v5"
scheduler = DDPMScheduler.from_pretrained(path)
model = UNet2DModel.from_pretrained(path).to("cuda")

pipe = StableDiffusionPipeline.from_pretrained(path, torch_dtype=torch.float16, safety_checker=None)
pipe = pipe.to("cuda")

In [ ]:
prompt = "((masterpiece, best quality, ultra-detailed)), close-up, face focus, upper body, blur background, girl, teeneger, school uniform, white shirt, brown hair, medium hair, hair clips, rose eyes, white eyeglasses, cheerful, smart, book"
ne_prompt = "3d, wet, sailor uniform, pilot uniform, blazer, maid, chest, big chest, tie, very long hair"

In [ ]:

for i in [0,1,2,4,6,8,16,32,64,128,256,512,1024,2048,3072,4096]:
    seed = i
    images1 = pipe(prompt,
                    guidance_scale=9,
                    height=640,
                    width=640,
                    generator=torch.Generator("cuda").manual_seed(seed),
                    negative_prompt= ne_prompt,
                    num_inference_steps=200,
            ).images
    images1[0].save('boymaganeblue/boyblue_{}.jpg'.format(seed))

In [ ]:
class Anything5(StableDiffusionPipeline):

  def __init__(self):
    self.guidance_scale = 9
    self.height = 640
    self.width = 640
    self.generator=torch.Generator("cuda").manual_seed(0)
    self.num_inference_steps=200

  def forward(self, prompt: str, ne_prompt: str = ""):
    return super().forward(prompt,
                  guidance_scale = self.guidance_scale,
                  height = self.height,
                  width = self.width,
                  generator = self.generator,
                  num_inference_steps = self.num_inference_steps,
                  ne_prompt = ne_prompt)


In [ ]:
device = torch.device('cpu')

model = Anything5()
model.from_pretrained("stablediffusionapi/anything-v5", torch_dtype=torch.float16, safety_checker=None)

model.eval()
model = model.to(device)

scripted_model = torch.jit.script(model)
optimized_model = optimize_for_mobile(scripted_model)
optimized_model.save('anything5.pt')